# Comprehensive DataFrame
The objective of this notebook is to analize the data that's being used in our database in order to see which statistics can be utilised.  
Lets start by importing the pandas library.

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xcc in position 4: invalid continuation byte  

So the csv file is not compatible with the normal UTF-8 encoding.  
I found the following information on the issue: [Unicode Error](https://stackoverflow.com/questions/18171739/unicodedecodeerror-when-reading-csv-file-in-pandas-with-python)  
By using ISO-8859-1 endoding instead of UTF-8, I was able to import the csv file as a pandas dataframe.

Added file to project git

In [90]:
import pandas as pd

driverStandingsCSV = 'driverStandings.csv'
driverStandingsDF  = pd.read_csv(driverStandingsCSV, header=0, index_col=None, encoding = "ISO-8859-1")
driverStandingsDF.head()

,driverStandingsId,raceId,driverId,points,position,positionText,wins
0,1,18,1,10.0,1,1,1
1,2,18,2,8.0,2,2,0
2,3,18,3,6.0,3,3,0
3,4,18,4,5.0,4,4,0
4,5,18,5,4.0,5,5,0


Looking at the data from the Driver Standings dataset, we can see that it does not really make a lot of sence on it's own.  
We'll need to add the information from other datasets and use it in unison with this dataset.  
We have the driversid which we can get more information on from the driver dataset. 
I can also see the raceid, which we can get more information on from the races dataset.
So let's start by also importing the these two datasets.

In [91]:
print("Drivers dataset")
driversCSV = 'drivers.csv'
driversDF  = pd.read_csv(driversCSV, header=0, index_col=None, encoding = "ISO-8859-1")
driversDF.head(3)

Drivers dataset


,driverId,driverRef,number,code,forename,surname,dob,nationality,url
0,1,hamilton,44.0,HAM,Lewis,Hamilton,07/01/1985,British,http://en.wikipedia.org/wiki/Lewis_Hamilton
1,2,heidfeld,NaN,HEI,Nick,Heidfeld,10/05/1977,German,http://en.wikipedia.org/wiki/Nick_Heidfeld
2,3,rosberg,6.0,ROS,Nico,Rosberg,27/06/1985,German,http://en.wikipedia.org/wiki/Nico_Rosberg


In [92]:
print("Races dataset")
racesCSV = 'races.csv'
racesDF  = pd.read_csv(racesCSV, header=0, index_col=None, encoding = "ISO-8859-1")

racesDF.head(3)

Races dataset


,raceId,year,round,circuitId,name,date,time,url
0,1,2009,1,1,Australian Grand Prix,2009-03-29,06:00:00,http://en.wikipedia.org/wiki/2009_Australian_G...
1,2,2009,2,2,Malaysian Grand Prix,2009-04-05,09:00:00,http://en.wikipedia.org/wiki/2009_Malaysian_Gr...
2,3,2009,3,17,Chinese Grand Prix,2009-04-19,07:00:00,http://en.wikipedia.org/wiki/2009_Chinese_Gran...


We can now start to combine data from the three datasets and construct a new dataset that is more comprehensive.  
Let's start by combining the driver standings and drivers dataframes based on the driver id.  
We can also drop the column used for merging as we'll no longer need this information, as well as the url column.

In [100]:
df1 = pd.merge(driverStandingsDF, driversDF, on="driverId")
df1 = df1.drop(columns=["url", "driverId", "driverStandingsId"])
df1.head()

,raceId,points,position,positionText,wins,driverRef,number,code,forename,surname,dob,nationality
0,18,10.0,1,1,1,hamilton,44.0,HAM,Lewis,Hamilton,07/01/1985,British
1,19,14.0,1,1,1,hamilton,44.0,HAM,Lewis,Hamilton,07/01/1985,British
2,20,14.0,3,3,1,hamilton,44.0,HAM,Lewis,Hamilton,07/01/1985,British
3,21,20.0,2,2,1,hamilton,44.0,HAM,Lewis,Hamilton,07/01/1985,British
4,22,28.0,3,3,1,hamilton,44.0,HAM,Lewis,Hamilton,07/01/1985,British


As we can see above, we no longer have the driver id, but rather the drivers name and surname, which makes for a more, human readible dataset.  
Let's go futher and add the races dataset to our comprehensive dataframe.  
We'll again delete the column used for merging, as well as url columns if any.

In [101]:
df2 = pd.merge(df1, racesDF, on="raceId")
df2 = df2.drop(columns=["url", "raceId"])
df2.head()

,points,position,positionText,wins,driverRef,number,code,forename,surname,dob,nationality,year,round,circuitId,name,date,time
0,10.0,1,1,1,hamilton,44.0,HAM,Lewis,Hamilton,07/01/1985,British,2008,1,1,Australian Grand Prix,2008-03-16,04:30:00
1,8.0,2,2,0,heidfeld,NaN,HEI,Nick,Heidfeld,10/05/1977,German,2008,1,1,Australian Grand Prix,2008-03-16,04:30:00
2,6.0,3,3,0,rosberg,6.0,ROS,Nico,Rosberg,27/06/1985,German,2008,1,1,Australian Grand Prix,2008-03-16,04:30:00
3,5.0,4,4,0,alonso,14.0,ALO,Fernando,Alonso,29/07/1981,Spanish,2008,1,1,Australian Grand Prix,2008-03-16,04:30:00
4,4.0,5,5,0,kovalainen,NaN,KOV,Heikki,Kovalainen,19/10/1981,Finnish,2008,1,1,Australian Grand Prix,2008-03-16,04:30:00


Above we can see that we now have a very comprehensive dataframe, showing the drivers name, surname, as well as information related to the races.  
We still have a circuitId column, which is a foreign key that refers to another dataset, so lets see if we can import that dataset and then include in our comprehensive dataframe.

In [102]:
circuitsCSV = 'circuits.csv'
circuitsDF  = pd.read_csv(circuitsCSV, header=0, index_col=None, encoding = "ISO-8859-1")
circuitsDF.head()

,circuitId,circuitRef,name,location,country,lat,lng,alt,url
0,1,albert_park,Albert Park Grand Prix Circuit,Melbourne,Australia,-37.84970,144.96800,10.0,http://en.wikipedia.org/wiki/Melbourne_Grand_P...
1,2,sepang,Sepang International Circuit,Kuala Lumpur,Malaysia,2.76083,101.73800,NaN,http://en.wikipedia.org/wiki/Sepang_Internatio...
2,3,bahrain,Bahrain International Circuit,Sakhir,Bahrain,26.03250,50.51060,NaN,http://en.wikipedia.org/wiki/Bahrain_Internati...
3,4,catalunya,Circuit de Barcelona-Catalunya,MontmelÌ_,Spain,41.57000,2.26111,NaN,http://en.wikipedia.org/wiki/Circuit_de_Barcel...
4,5,istanbul,Istanbul Park,Istanbul,Turkey,40.95170,29.40500,NaN,http://en.wikipedia.org/wiki/Istanbul_Park


Let's merge the two dataframes and also drop the columns from the circuits dataframe that we dont need.  
It should also be noted that we have a column named "name" in both the circuit and comprehensive dataframe, so lets see if we can rename the circuit name column.

In [103]:
circuitsDF.rename(columns={"name":"CircuitName"}, inplace=True)
circuitsDF.head()

,circuitId,circuitRef,CircuitName,location,country,lat,lng,alt,url
0,1,albert_park,Albert Park Grand Prix Circuit,Melbourne,Australia,-37.84970,144.96800,10.0,http://en.wikipedia.org/wiki/Melbourne_Grand_P...
1,2,sepang,Sepang International Circuit,Kuala Lumpur,Malaysia,2.76083,101.73800,NaN,http://en.wikipedia.org/wiki/Sepang_Internatio...
2,3,bahrain,Bahrain International Circuit,Sakhir,Bahrain,26.03250,50.51060,NaN,http://en.wikipedia.org/wiki/Bahrain_Internati...
3,4,catalunya,Circuit de Barcelona-Catalunya,MontmelÌ_,Spain,41.57000,2.26111,NaN,http://en.wikipedia.org/wiki/Circuit_de_Barcel...
4,5,istanbul,Istanbul Park,Istanbul,Turkey,40.95170,29.40500,NaN,http://en.wikipedia.org/wiki/Istanbul_Park


In [97]:
df3 = pd.merge(df2, circuitsDF, on="circuitId")
df3 = df3.drop(columns=["url", "alt", "circuitId"])
df3.head()

,points,position,positionText,wins,driverRef,number,code,forename,surname,dob,...,round,name,date,time,circuitRef,CircuitName,location,country,lat,lng
0,10.0,1,1,1,hamilton,44.0,HAM,Lewis,Hamilton,07/01/1985,...,1,Australian Grand Prix,2008-03-16,04:30:00,albert_park,Albert Park Grand Prix Circuit,Melbourne,Australia,-37.8497,144.968
1,8.0,2,2,0,heidfeld,NaN,HEI,Nick,Heidfeld,10/05/1977,...,1,Australian Grand Prix,2008-03-16,04:30:00,albert_park,Albert Park Grand Prix Circuit,Melbourne,Australia,-37.8497,144.968
2,6.0,3,3,0,rosberg,6.0,ROS,Nico,Rosberg,27/06/1985,...,1,Australian Grand Prix,2008-03-16,04:30:00,albert_park,Albert Park Grand Prix Circuit,Melbourne,Australia,-37.8497,144.968
3,5.0,4,4,0,alonso,14.0,ALO,Fernando,Alonso,29/07/1981,...,1,Australian Grand Prix,2008-03-16,04:30:00,albert_park,Albert Park Grand Prix Circuit,Melbourne,Australia,-37.8497,144.968
4,4.0,5,5,0,kovalainen,NaN,KOV,Heikki,Kovalainen,19/10/1981,...,1,Australian Grand Prix,2008-03-16,04:30:00,albert_park,Albert Park Grand Prix Circuit,Melbourne,Australia,-37.8497,144.968


We now have a comprehensive dataframe, with 22 columns, which is difficult to work with, but lets see which columns we have.

In [98]:
df3.columns

Index(['points', 'position', 'positionText', 'wins', 'driverRef', 'number',
       'code', 'forename', 'surname', 'dob', 'nationality', 'year', 'round',
       'name', 'date', 'time', 'circuitRef', 'CircuitName', 'location',
       'country', 'lat', 'lng'],
      dtype='object')

In [99]:
df3 = df3.sort_values(by='wins', ascending=False)

# Lets see the top 10 wins
startRow = df3.index[0]
endRow = df3.index[10]
df3.loc[startRow:endRow, ['wins', 'forename', 'surname', 'dob', 'nationality', 'year', 'CircuitName', 'date']]

,wins,forename,surname,dob,nationality,year,CircuitName,date
14464,13,Michael,Schumacher,03/01/1969,German,2004,AutÌ_dromo JosÌ© Carlos Pace,2004-10-24
14289,13,Sebastian,Vettel,03/07/1987,German,2013,AutÌ_dromo JosÌ© Carlos Pace,2013-11-24
17675,13,Michael,Schumacher,03/01/1969,German,2004,Suzuka Circuit,2004-10-10
14118,12,Michael,Schumacher,03/01/1969,German,2004,Shanghai International Circuit,2004-09-26
10895,12,Michael,Schumacher,03/01/1969,German,2004,Autodromo Nazionale di Monza,2004-09-12
18643,12,Sebastian,Vettel,03/07/1987,German,2013,Circuit of the Americas,2013-11-17
8199,12,Michael,Schumacher,03/01/1969,German,2004,Hungaroring,2004-08-15
9139,12,Michael,Schumacher,03/01/1969,German,2004,Circuit de Spa-Francorchamps,2004-08-29
18274,11,Sebastian,Vettel,03/07/1987,German,2011,Yas Marina Circuit,2011-11-13
14239,11,Sebastian,Vettel,03/07/1987,German,2011,AutÌ_dromo JosÌ© Carlos Pace,2011-11-27


## Conclusion
Now that we have a comprehensive dataframe, we can see that there's a lot of data that can be processed and a lot of statistical information that can be extracted.